# 1. Doodle Scheduling [10 pts]

#### by Roumen Guha, on Sunday, February 19th, 2017

Doodle Inc. is looking to interview a candidate for a new software engineer position at their company. It works like this: the interview (10 AM to 3 PM) is divided into a number of 20-minute time slots that may be used for 1-on-1 meetings with the candidate. There is also a one-hour time slot in the middle of the day where 3 employees take the candidate out for lunch.
It would be nice for all 15 senior employees to meet with the candidate at some point during the day, but everybody has a busy schedule so it's not clear whether this will be possible. A doodle poll (obviously) was sent to the 15 senior employees to figure out their availability.

<img src="1.png">

In the table, a 1 means that the employee is available at the indicated time, while a 0 means that they are unavailable. Determine whether a feasible interview schedule exists. If so, print out a calendar for the candidate that lists who they will be meeting at each time slot.

In [6]:
using NamedArrays      

# import data set
raw = readcsv("1b.csv")       # A file I made 
(m, n) = size(raw)

n_times = 2:n      # columns containing times
n_names = 2:m      # rows containing names

times = raw[1, n_times][:]   # the list of times (convert to 1-D array)
names = raw[n_names, 1][:]   # the list of names (convert to 1-D array)

# data[f,i] is the entire schedule
data = NamedArray(raw[n_names, n_times], (names, times), ("names", "times"))

show(IOContext(STDOUT, displaysize=(100, 1000)), data)

15×15 Named Array{Any,2}
names ╲ times │ 10:00  10:20  10:40  11:00  11:20  11:40  12:00  12:20  12:40  13:00  13:20  13:40  14:00  14:20  14:40
──────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────
Manuel        │     0      0      1      1      0      0      0      0      0      1      1      0      0      0      0
Luca          │     0      1      1      0      0      0      0      0      0      0      1      1      0      0      0
Jule          │     0      0      0      1      1      0      1      1      1      1      0      1      1      1      1
Michael       │     0      0      0      1      1      1      1      1      1      1      1      1      1      1      0
Malte         │     0      0      0      0      0      0      1      1      1      1      1      0      0      0      0
Chris         │     0      1      1      0      0      0      0      0      0      0      1      1      0      0      0
Spyros        │

In [7]:
using JuMP

m = Model()

@variable(m, x[names, times] >= 0)

# the candidate can only meet with 1 employee at any 20-minute period
@constraint(m, a[j in times], sum(x[i,j] for i in names) <= 1)
            
# each employee can only meet with the candidate once
@constraint(m, b[i in names], sum(x[i,j] for j in times) <= 1)
            
@objective(m, Max, sum(x[i,j]*data[i,j] for i in names, j in times))
                        
status = solve(m)
println(status)
                                    
schedule = NamedArray( [Int(getvalue(x[i,j])) for i in names, j in times], (names, times), ("Names", "Times"))
                        
show(IOContext(STDOUT, displaysize=(100, 1000)), schedule)

Optimal
15×15 Named Array{Int64,2}
Names ╲ Times │ 10:00  10:20  10:40  11:00  11:20  11:40  12:00  12:20  12:40  13:00  13:20  13:40  14:00  14:20  14:40
──────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────
Manuel        │     0      0      1      0      0      0      0      0      0      0      0      0      0      0      0
Luca          │     0      1      0      0      0      0      0      0      0      0      0      0      0      0      0
Jule          │     0      0      0      0      0      0      1      0      0      0      0      0      0      0      0
Michael       │     0      0      0      0      0      0      0      1      0      0      0      0      0      0      0
Malte         │     0      0      0      0      0      0      0      0      1      0      0      0      0      0      0
Chris         │     0      0      0      0      0      0      0      0      0      0      0      1      0      0      0
Spyro

#### Note:

To account for the lunch period, I simply split the lunch hour into 3 separate 20-minute periods with identical data.

In [9]:
println("Schedule is as follows: ")
println()

for j = 1:15
    for i = 1:15
        if schedule[i,j] == 1
            print(times[j])
            print(": ")
            println(names[i])
        end
    end
end

Schedule is as follows: 

10:00: Joel
10:20: Luca
10:40: Manuel
11:00: Daniel
11:20: Spyros
11:40: Anne
12:00: Jule
12:20: Michael
12:40: Malte
13:00: Josep
13:20: Florian
13:40: Chris
14:00: Matt
14:20: Mirjam
14:40: Tom


This means that Jule, Michael and Malte get to join the candidate for lunch. Lucky them.

In [11]:
println(m)

Max x[Manuel,10:40] + x[Manuel,11:00] + x[Manuel,13:00] + x[Manuel,13:20] + x[Luca,10:20] + x[Luca,10:40] + x[Luca,13:20] + x[Luca,13:40] + x[Jule,11:00] + x[Jule,11:20] + x[Jule,12:00] + x[Jule,12:20] + x[Jule,12:40] + x[Jule,13:00] + x[Jule,13:40] + x[Jule,14:00] + x[Jule,14:20] + x[Jule,14:40] + x[Michael,11:00] + x[Michael,11:20] + x[Michael,11:40] + x[Michael,12:00] + x[Michael,12:20] + x[Michael,12:40] + x[Michael,13:00] + x[Michael,13:20] + x[Michael,13:40] + x[Michael,14:00] + x[Michael,14:20] + x[Malte,12:00] + x[Malte,12:20] + x[Malte,12:40] + x[Malte,13:00] + x[Malte,13:20] + x[Chris,10:20] + x[Chris,10:40] + x[Chris,13:20] + x[Chris,13:40] + x[Spyros,11:00] + x[Spyros,11:20] + x[Spyros,11:40] + x[Spyros,12:00] + x[Spyros,12:20] + x[Spyros,12:40] + x[Mirjam,10:00] + x[Mirjam,10:20] + x[Mirjam,14:00] + x[Mirjam,14:20] + x[Mirjam,14:40] + x[Matt,10:00] + x[Matt,10:20] + x[Matt,10:40] + x[Matt,13:40] + x[Matt,14:00] + x[Florian,13:00] + x[Florian,13:20] + x[Josep,12:00] + x[Jos